<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/enhanced_multiverse_prediction_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
enhanced_multiverse_prediction.py

Train a deep neural network to predict multiverse topology classes
from synthetic string-theory compactification parameters.
Improvements include class-balanced loss, batchnorm, dropout,
ReduceLROnPlateau, early stopping, and checkpointing.
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report

# 1. Synthetic Dataset Generation
def sample_compactifications(n_samples=15000, input_dim=5):
    """
    Generate synthetic compactification parameters and discrete labels.
    Uses np.ptp() for NumPy 2.0 compatibility.
    """
    params = np.random.uniform(0, 2*np.pi, size=(n_samples, input_dim)).astype(np.float32)
    scores = np.sum(np.sin(params), axis=1)
    span = np.ptp(scores) + 1e-6
    labels = np.floor((scores - scores.min()) / span * 10).astype(int)
    labels = np.clip(labels, 0, 9)
    return params, labels

# 2. Model Definition
class MultiverseAI(nn.Module):
    def __init__(self, input_dim=5, hidden_dims=[64, 32], output_dim=10, drop_p=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dims[0])
        self.bn1 = nn.BatchNorm1d(hidden_dims[0])
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.bn2 = nn.BatchNorm1d(hidden_dims[1])
        self.fc3 = nn.Linear(hidden_dims[1], output_dim)
        self.drop = nn.Dropout(drop_p)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.drop(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.drop(x)
        return F.softmax(self.fc3(x), dim=-1)

# 3. Training & Validation Loop
def train_and_validate(
    model, train_loader, val_loader,
    class_weights, epochs=100, lr=1e-3, patience=10, device=None
):
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Weighted cross-entropy
    weight_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)
    criterion = nn.CrossEntropyLoss(weight=weight_tensor)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                     mode="max",
                                                     factor=0.5,
                                                     patience=5,
                                                     verbose=True)

    best_val_acc = 0.0
    epochs_no_improve = 0
    all_true, all_preds = [], []

    for epoch in range(1, epochs + 1):
        # Training
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * xb.size(0)
            correct_train += (preds.argmax(1) == yb).sum().item()
            total_train += xb.size(0)

        train_loss = running_loss / total_train
        train_acc = correct_train / total_train

        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        epoch_true, epoch_pred = [], []

        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                correct_val += (preds.argmax(1) == yb).sum().item()
                total_val += xb.size(0)

                epoch_true.append(yb.cpu().numpy())
                epoch_pred.append(preds.argmax(1).cpu().numpy())

        val_acc = correct_val / total_val
        scheduler.step(val_acc)

        # Checkpointing & Early Stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
            torch.save(model.state_dict(), "best_multiverse_ai.pth")
        else:
            epochs_no_improve += 1

        # Log progress
        if epoch == 1 or epoch % 10 == 0 or val_acc == best_val_acc:
            print(f"Epoch {epoch:03d} ─ "
                  f"Train Loss: {train_loss:.4f} ─ "
                  f"Train Acc: {train_acc:.3f} ─ "
                  f"Val Acc: {val_acc:.3f}")

        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch}. "
                  f"Best Val Acc: {best_val_acc:.3f}")
            all_true = np.concatenate(epoch_true)
            all_preds = np.concatenate(epoch_pred)
            break

        all_true = np.concatenate(epoch_true)
        all_preds = np.concatenate(epoch_pred)

    # Final metrics
    print("\nClassification Report:")
    print(classification_report(all_true, all_preds, digits=4))
    print("Confusion Matrix:")
    print(confusion_matrix(all_true, all_preds))

# 4. Main Execution
if __name__ == "__main__":
    # Hyperparameters
    INPUT_DIM   = 5
    OUTPUT_DIM  = 10
    N_SAMPLES   = 15000
    TEST_SIZE   = 0.2
    BATCH_SIZE  = 256
    EPOCHS      = 100
    LR          = 1e-3
    PATIENCE    = 10

    # Generate and balance dataset
    X, y = sample_compactifications(N_SAMPLES, INPUT_DIM)
    classes, counts = np.unique(y, return_counts=True)
    inv_counts = 1.0 / (counts + 1e-6)
    class_weights = inv_counts / inv_counts.sum() * len(inv_counts)

    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=42
    )

    # Create DataLoaders
    train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    val_ds   = TensorDataset(torch.from_numpy(X_val),   torch.from_numpy(y_val))
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

    # Initialize model
    model = MultiverseAI(
        input_dim=INPUT_DIM,
        hidden_dims=[64, 32],
        output_dim=OUTPUT_DIM,
        drop_p=0.3
    )

    # Train and validate
    train_and_validate(
        model, train_loader, val_loader,
        class_weights, epochs=EPOCHS, lr=LR, patience=PATIENCE
    )